In [1]:
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import random
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfTransformer
import gc

TEST_SET_THRESHOLD = 10
TEST_SET_HOLDOUT = 0.2
BEST_ALFA = 0.92

In [3]:
import utils

In [4]:
tracks = pd.read_csv('../input/tracks.csv')
train = pd.read_csv('../input/train.csv')
target = pd.read_csv('../input/target_playlists.csv')

In [5]:
icm_csr = utils.build_icm_csr(tracks)
urm_csr = utils.build_urm_csr(train)

/home/antonio/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py:746: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [6]:
urm_csr = utils.build_urm_csr(train)

In [7]:
training_set, test_set = utils.split(train,TEST_SET_HOLDOUT=TEST_SET_HOLDOUT,TEST_SET_THRESHOLD=TEST_SET_THRESHOLD)

In [8]:
urm_train_csr = utils.build_urm_csr(training_set)
urm_test_csr = utils.build_urm_csr(test_set)

In [9]:
urm_test_csr

<50446x20635 sparse matrix of type '<class 'numpy.float32'>'
	with 77570 stored elements in Compressed Sparse Row format>

In [10]:
urm_train_csr

<50446x20635 sparse matrix of type '<class 'numpy.float32'>'
	with 1134221 stored elements in Compressed Sparse Row format>

In [11]:
test_playlists = test_set['playlist_id'].unique()

In [12]:
from Base.Evaluation.Evaluator import SequentialEvaluator

In [13]:
batches = [1,2,5,10]
ks = [10,25,50,75,100]
MAPS = []
for batch in batches:
    for k in ks:
        print("TRAINING with {} batchs and top {}".format(batch,k))
        evaluator = SequentialEvaluator(urm_test_csr,cutoff_list=[10])
        recommender = SLIM_BPR_Cython(urm_train_csr,recompile_cython=False,positive_threshold=1) 
        recommender.fit(epochs=30,batch_size=5,sgd_mode='adam',learning_rate=1e-4,topK=80)
        gc.collect()
        datas,_ = evaluator.evaluateRecommender(recommender)
        MAP = datas[10]['MAP']
        MAPS.append([batch,k,MAP])
        print("*****RESULT with {} batchs and top {} is MAP = {}".format(batch,k,MAP))
        gc.collect()

TRAINING with 1 batchs and top 10
SLIM_BPR_Cython: Estimated memory required for similarity matrix of 20635 items is 1703.21 MB
Processed 1134221 ( 100.00% ) in 4.48 seconds. BPR loss is 2.31E-02. Sample per second: 253439
SLIM_BPR_Recommender: Epoch 1 of 30. Elapsed time 0.12 min
Processed 1134221 ( 100.00% ) in 4.36 seconds. BPR loss is 8.53E-02. Sample per second: 259847
SLIM_BPR_Recommender: Epoch 2 of 30. Elapsed time 0.19 min
Processed 1134221 ( 100.00% ) in 4.20 seconds. BPR loss is 1.59E-01. Sample per second: 270083
SLIM_BPR_Recommender: Epoch 3 of 30. Elapsed time 0.25 min
Processed 1134221 ( 100.00% ) in 3.98 seconds. BPR loss is 2.39E-01. Sample per second: 285239
SLIM_BPR_Recommender: Epoch 4 of 30. Elapsed time 0.32 min
Processed 1134221 ( 100.00% ) in 4.84 seconds. BPR loss is 3.27E-01. Sample per second: 234207
SLIM_BPR_Recommender: Epoch 5 of 30. Elapsed time 0.38 min
Processed 1134221 ( 100.00% ) in 4.75 seconds. BPR loss is 4.27E-01. Sample per second: 238923
SLIM_BP

KeyboardInterrupt: 

In [ ]:
target = target.get_values()[:,0]

In [ ]:
n = 500
  
# using list comprehension 
divided_target = [target[i * n:(i + 1) * n] for i in range((len(target) + n - 1) // n )]  

In [ ]:
result = []
for items in target:
    partial_recomendations = recommender.recommend(items,cutoff=10)
    result.append(partial_recomendations)

In [ ]:
result = np.array(result)

In [ ]:
result.reshape(-1,10)

In [ ]:
result = result.reshape(-1,10)

In [ ]:
result.shape

In [ ]:
i = 0
results = []
for item in result:
    recommended_items = " ".join(str(i) for i in item)
    temp = [target[i],recommended_items]
    results.append(temp)
    i += 1

In [ ]:
rec = pd.DataFrame(results)
rec.to_csv("submission_new.csv", index = False, header = ["playlist_id", "track_ids"])

In [ ]:
MAPS

In [14]:
gc.collect()

9

In [21]:
recommender.

<20635x20635 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>